# mjlog 解析 sketchbook

## 基準

- 配牌
    - P1: hai0="112,124,35,135,10,5,25,85,66,8,43,19,1,33"
    - P2: hai1="46,96,73,53,128,120,98,89,28,49,41,83,72"
- ツモ 
    - P0: <T[0-9]>
    - P1: <U[0-9]> 
    - P2: <V[0-9]>
    - P3: <W[0-9]>
- 打牌 
    - P0: <D[0-9]>
    - P1: <E[0-9]>
    - P2: <F[0-9]>
    - P3: <G[0-9]>

In [5]:
import glob
import gzip
import os
import re
import xml.etree.ElementTree as ET

from bs4 import BeautifulSoup
import numpy as np
from tqdm import notebook as tqdm

from meld import Meld

### 捨て牌モデル学習用データの作成

カラムは15列(配牌+ツモの14枚 と 切った牌1枚) をone-hotに変換する 34 * 14 → 34になるようにする。

### ToDo

- 牌譜からツモと切る対応を作る
    - 途中の副露に対応する



In [2]:
def parse_mjlog_to_xml(path):
    with gzip.open(path, mode='rt') as fp:
        mjlog_text = fp.read()
    mjlog_xml = ET.fromstring(mjlog_text)

    return mjlog_text, mjlog_xml

def get_discard_other_pai_vector(manage_each_sutehai_list, player_num):
    other_num_list = [1,2,3] if player_num == 0 else [0,2,3] if player_num == 1 else [0,1,3] if player_num == 2 else [0,1,2]
    other_player_discard_vector_list = []

    manage_each_sutehai_list[player_num].append(pai)
    for player in other_num_list:
        other_player_discard_vector_list.append(
            np.identity(34)[manage_each_sutehai_list[player]].tolist() \
                + np.zeros((24 - len(manage_each_sutehai_list[player]), 34)).tolist()
        )
    
    return manage_each_sutehai_list, other_player_discard_vector_list

In [3]:
mjlog_path_list = sorted(glob.glob('mjlog_pf4-20_n1/**.mjlog'))[:10]

In [4]:
# ToDo: 定期的にエクスポートするコードを書く
player_discard_train_vector_list = []
other_discard_train_vector_list = []
player_discard_target_vector_list= []
 
for mjlog_path in tqdm(mjlog_path_list):
    mjlog_text, mjlog_xml = parse_mjlog_to_xml(os.path.join(os.getcwd(), mjlog_path))
    # 1ゲーム
    for n, (mj_xml, mj_text) in enumerate(zip(mjlog_xml.findall('INIT'), re.findall("<INIT [^>]*?>(.+?)(<AGARI [^>]*?|<RYUUKYOKU [^>]*?>)", mjlog_text))):
        manage_each_tehai_list = [
            sorted([int(pai_num) // 4 for pai_num in mj_xml.get('hai0').split(',')]),
            sorted([int(pai_num) // 4 for pai_num in mj_xml.get('hai1').split(',')]),
            sorted([int(pai_num) // 4 for pai_num in mj_xml.get('hai2').split(',')]),
            sorted([int(pai_num) // 4 for pai_num in mj_xml.get('hai3').split(',')])
        ]
        manage_each_sutehai_list = [[] for _ in range(4)]
        # 1局
        for move in mj_text[0].split('/>')[:-1]:
            # 牌を捨てるタイミングでデータセットを作成する
            if move[1:4] in ('REA', 'BYE', 'UN ', 'DOR'): # REACH, BYE, UN , DORAを除外
                continue

            player_num = \
                0 if move[1] in ('T', 'D') else \
                1 if move[1] in ('U', 'E') else \
                2 if move[1] in ('V', 'F') else \
                3 if move[1] in ('W', 'G') else None 

            # ツモるもしくは捨てる場合は選択牌を管理
            if move[1] != 'N':
                pai = int(move[2:]) // 4
            
            # ツモる
            if move[1] in ('T', 'U', 'V', 'W'):
                manage_each_tehai_list[player_num].append(pai)
            
            # 捨てる
            if move[1] in ('D', 'E', 'F', 'G'):
                # onehot表現に変換してtrainリストへ
                tehai_onehot_vector = np.identity(34)[manage_each_tehai_list[player_num]].tolist()
                # 副露対応
                if len(tehai_onehot_vector) < 14:
                    while len(tehai_onehot_vector) < 14:
                        tehai_onehot_vector.append(np.zeros(34).tolist())

                # 他家の捨て牌
                manage_each_sutehai_list, other_player_discard_vector_list = \
                    get_discard_other_pai_vector(manage_each_sutehai_list, player_num)
                other_discard_train_vector_list.append(other_player_discard_vector_list)

                # 自分の手牌
                player_discard_train_vector_list.append(sorted(tehai_onehot_vector))

                # 自分の切り出し牌
                player_discard_target_vector_list.append(np.identity(34)[pai].tolist())

                # 手牌操作
                manage_each_tehai_list[player_num].remove(pai)

            # 鳴く
            if move[1] == 'N':
                meld_info = move.split(' ')
                who = int(re.sub("\\D", "", meld_info[1]))
                meld_num = int(re.sub("\\D", "", meld_info[2]))
                meld_info_list = Meld(meld_num).getIndex()

                if meld_info_list in ('ポン', 'チー'):
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[3])
                elif meld_info_list == '加カン':
                    manage_each_tehai_list[who].remove(meld_info_list[4])
                elif meld_info_list == '明カン':
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                elif meld_info_list == '暗カン':
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[2])
                    manage_each_tehai_list[who].remove(meld_info_list[2])


<ipython-input-4-fe3b6f95cae2>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for mjlog_path in tqdm(mjlog_path_list):


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
manage_each_tehai_list = [haipai_p0.copy(), haipai_p1.copy(), haipai_p2.copy(), haipai_p3.copy()]
manage_each_sutehai_list = [[] for _ in range(4)]
player_discard_train_vector_list = []
other_discard_train_vector_list = []
player_discard_target_vector_list= []

for move in tqdm(sutehai_all.split('/>')[:-1]):
    train_data_list = []
    # 牌を捨てるタイミングでデータセットを作成する
    if move[1] == 'R': # REACH
        continue

    player_num = \
        0 if move[1] in ('T', 'D') else \
        1 if move[1] in ('U', 'E') else \
        2 if move[1] in ('V', 'F') else \
        3 if move[1] in ('W', 'G') else None 

    # ツモるもしくは捨てる場合は選択牌を管理
    if move[1] != 'N':
        pai = int(move[2:]) // 4
    
    # ツモる
    if move[1] in ('T', 'U', 'V', 'W'):
        manage_each_tehai_list[player_num].append(pai)
    
    # 捨てる
    if move[1] in ('D', 'E', 'F', 'G'):
        # onehot表現に変換してtrainリストへ
        tehai_onehot_vector = np.identity(34)[manage_each_tehai_list[player_num]].tolist()
        # 副露対応
        if len(tehai_onehot_vector) < 14:
            while len(tehai_onehot_vector) < 14:
                tehai_onehot_vector.append(np.zeros(34).tolist())

        # 他家の捨て牌
        manage_each_sutehai_list, other_player_discard_vector_list = \
            get_discard_other_pai_vector(manage_each_sutehai_list, player_num)
        other_discard_train_vector_list.append(other_player_discard_vector_list)

        # 自分の手牌
        player_discard_train_vector_list.append(sorted(tehai_onehot_vector))

        # 自分の切り出し牌
        player_discard_target_vector_list.append(np.identity(34)[pai].tolist())

        # 手牌操作
        manage_each_tehai_list[player_num].remove(pai)

    # 鳴く
    if move[1] == 'N':
        meld_info = move.split(' ')
        who = int(re.sub("\\D", "", meld_info[1]))
        meld_num = int(re.sub("\\D", "", meld_info[2]))
        meld_info_list = Meld(meld_num).getIndex()

        manage_each_tehai_list[who].remove(meld_info_list[2])
        manage_each_tehai_list[who].remove(meld_info_list[3])

In [5]:
np.array(other_discard_train_vector_list).shape

(5034, 3, 24, 34)

In [6]:
np.array(player_discard_train_vector_list).shape

<ipython-input-6-7e3d31361622>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(player_discard_train_vector_list).shape


(5034,)